<a href="https://colab.research.google.com/github/Otsemeuno/Customer-Subscription/blob/main/Customer_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Customer Subscription
$(Adapted from Kaggle)$


## Project Statement
This data is about a subscription-based digital product offering for financial advisory that includes newsletters, webinars, and investment recommendations. The offering has a couple of varieties, annual subscription, and digital subscription. The product also provides daytime support for customers to reach out to a care team that can help them with any product-related questions and signup/cancellation-related queries.

The data set contains the following information:

<ul><li>customer sign-up and cancellation dates at the product level
<li>call center activity
<li>customer demographics
<li>product pricing info<ul>

## Objectives

Forecast 2022 year revenue if the company expects to add the same number of new customers as the previous year

In 2022, one of the company's key initiatives is to improve the customer experience, one of the capabilities is superior customer care service, and they are thinking about enhancing in-product support better but that takes beyond 2022 to happen. It is very important to forecast call center case volume accurately so that they can make appropriate staffing decisions. Need to predict daily call volumes and make recommendations on how to staff on an hourly basis.

## Importing necessaries files

In [160]:
#importing drive
from google.colab import drive

In [161]:
#importing all necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import math

In [162]:
#suppress all warnings
import warnings
warnings.filterwarnings("ignore")

##Data Preparation

####Load the dataset

In [163]:
# Mount Google Drive on Kernel
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
customer_cases = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_cases.csv', index_col =[0])

In [165]:
customer_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_info.csv', index_col =[0])

In [166]:
customer_product = pd.read_csv('/content/drive/MyDrive/Kaggle project/customer_product.csv', index_col =[0])

In [167]:
product_info = pd.read_csv('/content/drive/MyDrive/Kaggle project/product_info.csv', index_col =[0])

In [168]:
#display top 5 rows in the dataset
customer_cases.head()

,case_id,date_time,customer_id,channel,reason
1,CC101,2017-01-01 10:32:03,C2448,phone,signup
2,CC102,2017-01-01 11:35:47,C2449,phone,signup
3,CC103,2017-01-01 11:37:09,C2450,phone,signup
4,CC104,2017-01-01 13:28:14,C2451,phone,signup
5,CC105,2017-01-01 13:52:22,C2452,phone,signup


In [169]:
#display top 5 rows in the dataset
customer_info.head()

,customer_id,age,gender
1,C2448,76,female
2,C2449,61,male
3,C2450,58,female
4,C2451,62,female
5,C2452,71,male


In [170]:
#display top 5 rows in the dataset
customer_product.head() 

,customer_id,product,signup_date_time,cancel_date_time
1,C2448,prd_1,2017-01-01 10:35:09,NaN
2,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02
3,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55
4,C2451,prd_2,2017-01-01 13:32:08,NaN
5,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01


In [171]:
#display top 5 rows in the dataset
product_info.head()

,name,price,billing_cycle
product_id,,,
prd_1,annual_subscription,1200,12
prd_2,monthly_subscription,125,1


In [172]:
customer_cases.shape

(330512, 5)

The customer cases data set has 5 attributes and 330512 observations

In [173]:
customer_info.shape

(508932, 3)

The data on customer info has 3 attributes and 508932 observations

In [174]:
customer_product.shape

(508932, 4)

Customer product has 508932 records and 4 features

In [175]:
product_info.shape

(2, 3)

Data provided has information on two products

###Missing Variables

In [176]:
customer_cases.isna().sum()

case_id        0
date_time      0
customer_id    0
channel        0
reason         0
dtype: int64

In [177]:
customer_info.isna().sum()

customer_id    0
age            0
gender         0
dtype: int64

In [178]:
customer_product.isna().sum()

customer_id              0
product                  0
signup_date_time         0
cancel_date_time    396447
dtype: int64

In [179]:
product_info.isna().sum()

name             0
price            0
billing_cycle    0
dtype: int64

Customer_produc has missing values. This missing values are from the cancel_date_time attribute. The values are missing because the customers are yet to cancel their current plan and are still with the company.

##Feature Engineering

In [180]:
customer_product['customer'] = customer_product.cancel_date_time.apply(lambda x: 1 if x is np.nan else 0)

In [181]:
customer_product.columns

Index(['customer_id', 'product', 'signup_date_time', 'cancel_date_time',
       'customer'],
      dtype='object')

## Forecasting Revenue

Merge the two dataset to get all  the prices of each product subscribed by the customer.

In [182]:
df_rev = customer_product.merge(product_info, left_on = 'product', right_on = 'product_id')

  Replace the null values with the last day of the year 2021

In [183]:
#Convert the signup date and cancellation dates to datetime
df_rev['signup_date_time'] = pd.to_datetime(df_rev['signup_date_time']).dt.date
df_rev['cancel_date_time'] = pd.to_datetime(df_rev['cancel_date_time']).dt.date

In [184]:
# Define the proposed date
proposed_date = pd.to_datetime('2021-12-31')

# Replace the NaT value with the proposed date
df_rev['cancel_date_time'] = df_rev['cancel_date_time'].fillna(proposed_date)


In [185]:
#get date from date time
df_rev['cancel_date_time'] = pd.to_datetime(df_rev['cancel_date_time']).dt.date

In [186]:
df_rev.head()

,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01,2021-09-05,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01,2019-01-13,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01,2021-06-28,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12


In [187]:
df_rev.head()

,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01,2021-09-05,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01,2019-01-13,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01,2021-06-28,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12


Separate the data into monthly subscription and yearly subscription

In [188]:
df_rev_month = df_rev[df_rev['name'] == 'monthly_subscription']

In [189]:
df_rev_year = df_rev[df_rev['name'] == 'annual_subscription']
df_rev_year.head()


,customer_id,product,signup_date_time,cancel_date_time,customer,name,price,billing_cycle
0,C2448,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12
1,C2449,prd_1,2017-01-01,2021-09-05,0,annual_subscription,1200,12
2,C2450,prd_1,2017-01-01,2019-01-13,0,annual_subscription,1200,12
3,C2452,prd_1,2017-01-01,2021-06-28,0,annual_subscription,1200,12
4,C2453,prd_1,2017-01-01,2021-12-31,1,annual_subscription,1200,12


### Revenue generated monthly from the monthly subscription

For the monthly subscription, we need to find the number of active months subscribed.

In [190]:
df_rev_month.loc[:, 'months'] = df_rev_month.apply(lambda row: len(pd.period_range(start=row['signup_date_time'], end=row['cancel_date_time'], freq='M')), axis=1)

In [199]:
# identify the months in the range of dataset
start_date = pd.to_datetime('2017-01-01')
end_date = pd.to_datetime('2021-12-31')
date_range = pd.date_range(start_date, end_date, freq = 'MS')
monthly_revenue = pd.DataFrame(index = date_range)
monthly_revenue.head()


""
2017-01-01
2017-02-01
2017-03-01
2017-04-01
2017-05-01


In [198]:
for i, month in enumerate(monthly_revenue.index):
  active_customers = df_rev_month[(df_rev_month['signup_date_time'] <= month) & 
                                  (df_rev_month['cancel_date_time'] >= month)]
  monthly_revenue.loc[month, 'Revenue'] = (active_customers['price']).sum()

monthly_revenue.head()

,Revenue
2017-01-01,750.0
2017-02-01,87625.0
2017-03-01,158125.0
2017-04-01,241000.0
2017-05-01,321875.0


### Revenue generated monthly from the annual subscription

Number of years subscribed for each customer 

In [193]:
df_rev_year['year'] = ((df_rev_year['cancel_date_time'] - df_rev_year['signup_date_time']).dt.days//365)
df_rev_year['year'] = df_rev_year['year'].apply(lambda x: math.ceil(x))

In [194]:
df_rev_year['signup_date_time'] = pd.to_datetime(df_rev_year['signup_date_time'])

In [195]:
yearly_revenue = pd.DataFrame(index=date_range)

In [196]:
for i, month in enumerate(monthly_revenue.index):
  active_customers_year = df_rev_year[(df_rev_year['signup_date_time'] <= month) & 
                                  (df_rev_year['cancel_date_time'] >= month) &
                                  (df_rev_year['signup_date_time'].dt.month == month.month)]
  yearly_revenue.loc[month, 'Revenue'] = (active_customers_year['price']).sum()

The total revenue generated in each month will be the sum of revenue generated for both the monthly and annual subscription.

In [197]:
total_month_rev = yearly_revenue + monthly_revenue
total_month_rev.head()

,Revenue
2017-01-01,36750.0
2017-02-01,241225.0
2017-03-01,346525.0
2017-04-01,295000.0
2017-05-01,492275.0
